In [19]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, geom, poisson

---
### Binomial Distribution


In [20]:
def simulate_binomial(n_trials, p, n_experiment, target):
    results = []
    for _ in range(n_experiments):
        flips = np.random.rand(n_trials) < p
        successes = flips.sum()
        results.append(successes)

    results = np.array(results)

    if target is not None:
        estimate = np.mean(results == target)
        print(f"Estimated P(X = {target}) ≈ {estimate:.4f}")
        return results, estimate

    return results

---
### Geometric Distribution

In [21]:
def simulate_geometric(p, n_experiments, target):
    results = []
    for _ in range(n_experiments):
        count = 1
        while np.random.rand() > p:
            count += 1
        results.append(count)

    results = np.array(results)

    if target is not None:
        estimate = np.mean(results == target)
        print(f"Estimated P(X = {target}) ≈ {estimate:.4f}")
        return results, estimate

    return results

---
### Poisson Distribution

In [22]:
def simulate_poisson(lam, interval_time, n_experiments, target):
    results = []
    for _ in range(n_experiments):
        time = 0
        count = 0
        while time < interval_time:
            interarrival = np.random.exponential(scale=1 / lam)
            time += interarrival
            if time < interval_time:
                count += 1
        results.append(count)

    results = np.array(results)

    if target is not None:
        estimate = np.mean(results == target)
        print(f"Estimated P(X = {target}) ≈ {estimate:.4f}")
        return results, estimate

    return results

---
### Visualizer

In [41]:
def plot_simulated_vs_theoretical(data, dist_name, pmf_func, x_range, title, **pmf_kwargs):
    plt.figure(figsize=(8, 5))
    
    # Plot histogram of simulation
    plt.hist(data, bins=np.arange(min(data), max(data) + 2) - 0.5,
             density=True, alpha=0.6, color='skyblue', edgecolor='black', label='Simulated')

    # Overlay theoretical PMF
    x = np.arange(x_range[0], x_range[1])
    pmf = pmf_func(x, **pmf_kwargs)
    plt.plot(x, pmf, 'o-', color='red', label='Theoretical PMF')

    # Labels and titles
    plt.title(f"{title} (Sample Size = {len(data)})")
    plt.xlabel('Value')
    plt.ylabel('Probability')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

---
### Analyzer

In [43]:
def analyze_distribution(data, dist_name, theoretical_mean, theoretical_var):
    empirical_mean = np.mean(data)
    empirical_var = np.var(data)
    
    print(f"--- {dist_name} Analysis ---")
    print(f"Theoretical Mean: {theoretical_mean}")
    print(f"Empirical Mean:   {empirical_mean:.4f}")
    print(f"Theoretical Variance: {theoretical_var}")
    print(f"Empirical Variance:   {empirical_var:.4f}")

    if empirical_mean > theoretical_mean:
        skew_comment = "Right-skewed (long tail to the right)"
    elif empirical_mean < theoretical_mean:
        skew_comment = "Left-skewed (long tail to the left)"
    else:
        skew_comment = "Symmetric"

    print(f"Shape Insight: {skew_comment}")
    print()

---
### Main Menu

In [45]:
def main_menu():
    print("Discrete Distribution Simulator")
    print("Select a distribution to simulate:\n")
    print("1. Binomial Distribution")
    print("2. Geometric Distribution")
    print("3. Poisson Distribution")
    print("4. Visualize and Compare (Simulated vs Theoretical) + Analysis")  # Updated description

    choice = input("Enter your choice (1/2/3/4): ")

    if choice == "1":
        print("\nBinomial Simulator")
        n_trials = int(input("Enter number of trials (n): "))
        p = float(input("Enter success probability (p): "))
        n_experiments = int(input("Enter number of simulations: "))
        target = int(input("Enter desired number of successes: "))
        simulate_binomial(n_trials, p, n_experiments, target)

    elif choice == "2":
        print("\nGeometric Simulator")
        p = float(input("Enter success probability (p): "))
        n_experiments = int(input("Enter number of simulations: "))
        target = int(input("Enter desired trial for first success: "))
        simulate_geometric(p, n_experiments, target)

    elif choice == "3":
        print("\nPoisson Simulator")
        lam = float(input("Enter average rate (λ): "))
        interval = float(input("Enter interval length (e.g., 1.0): "))
        n_experiments = int(input("Enter number of simulations: "))
        target = int(input("Enter desired number of events in interval: "))
        simulate_poisson(lam, interval, n_experiments, target)

    elif choice == "4":
        print("\nVisualizing all 3 distributions with overlays and running analysis...")
        sample_size = int(input("Enter sample size for simulation (e.g., 1000): "))

        # Binomial Inputs
        print("\n--- Binomial Distribution ---")
        n_binom = int(input("Enter number of trials (n): "))
        p_binom = float(input("Enter success probability (p): "))
        binom_data = np.random.binomial(n=n_binom, p=p_binom, size=sample_size)
        plot_simulated_vs_theoretical(
            binom_data,
            "Binomial",
            binom.pmf,
            x_range=(0, n_binom + 1),
            title=f"Binomial Distribution (n={n_binom}, p={p_binom})",
            n=n_binom,
            p=p_binom
        )
        analyze_distribution(
            binom_data,
            "Binomial",
            theoretical_mean=n_binom * p_binom,
            theoretical_var=n_binom * p_binom * (1 - p_binom)
        )

        # Geometric Inputs
        print("\n--- Geometric Distribution ---")
        p_geom = float(input("Enter success probability (p): "))
        geom_data = np.random.geometric(p=p_geom, size=sample_size)
        geom_max = min(np.max(geom_data) + 1, 30)
        plot_simulated_vs_theoretical(
            geom_data,
            "Geometric",
            geom.pmf,
            x_range=(1, geom_max),
            title=f"Geometric Distribution (p={p_geom})",
            p=p_geom
        )
        geom_mean = 1 / p_geom
        geom_var = (1 - p_geom) / (p_geom ** 2)
        analyze_distribution(
            geom_data,
            "Geometric",
            theoretical_mean=geom_mean,
            theoretical_var=geom_var
        )

        # Poisson Inputs
        print("\n--- Poisson Distribution ---")
        lam_poisson = float(input("Enter average rate (λ): "))
        k_target = int(input("Enter desired number of events (k): "))
        poisson_data = np.random.poisson(lam=lam_poisson, size=sample_size)
        upper_bound = min(int(np.max(poisson_data) + 5), int(lam_poisson * 3.5))
        plot_simulated_vs_theoretical(
            poisson_data,
            "Poisson",
            poisson.pmf,
            x_range=(0, upper_bound),
            title=f"Poisson Distribution (λ={lam_poisson})",
            mu=lam_poisson
        )
        prob_k = poisson.pmf(k_target, mu=lam_poisson)
        print(f"\nP(X = {k_target}) = {prob_k:.5f}")
        analyze_distribution(
            poisson_data,
            "Poisson",
            theoretical_mean=lam_poisson,
            theoretical_var=lam_poisson
        )
    
    else:
        print("Invalid choice. Please select 1, 2, 3, or 4.")

main_menu()

Discrete Distribution Simulator
Select a distribution to simulate:

1. Binomial Distribution
2. Geometric Distribution
3. Poisson Distribution
4. Visualize and Compare (Simulated vs Theoretical) + Analysis


Enter your choice (1/2/3/4):  5


Invalid choice. Please select 1, 2, 3, or 4.
